In [1]:
import sys
import os

# one level up from the Notebooks directory
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(notebook_dir)

# Import environment setup
import setup

# Now you can safely import your modules
from data_pipeline.pipeline import EnergyDemandForecasting
import pandas as pd


In [2]:
from config.config import FEATURE_ENGINEERED_DATA_PATH, INFERENCE_DATA_PATH, RAW_DATA_PATH

### For Day-ahead


In [6]:
ed = EnergyDemandForecasting()

#data = ed.featured_data_loading(FEATURE_ENGINEERED_DATA_PATH=FEATURE_ENGINEERED_DATA_PATH)
#data.head(2)

In [5]:
inf_data = pd.read_csv(RAW_DATA_PATH)
inf_data.head(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
0,2024-01-01 00:00:00,232.483571,5.124673,1.191402,0.399029,0.907766,46.614845,26.399566,0.0,1.055937,0,0,17.842767,64.523353,0.416156,1.233158


In [7]:
inf_data.tail(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
49999,2025-06-04 19:45:00,237.579055,17.904864,4.253821,1.050729,0.939617,7.220105,24.808317,0.0,-1.584342,0,0,15.083737,43.051294,0.360774,4.361657


In [13]:
inf_data['Timestamp'] = pd.to_datetime(inf_data['Timestamp'])  # ensure it's datetime
inf_data = inf_data.set_index('Timestamp') 
inf_data = inf_data[inf_data.columns].resample('D').mean()

inf_data.tail(3)

,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
Timestamp,,,,,,,,,,,,,,,
2025-06-02,229.421422,26.679700,6.122319,1.566531,0.900706,24.722421,16.057297,0.068809,-0.240128,0.104167,0.031250,24.733840,47.746748,0.301807,6.091351
2025-06-03,230.035298,27.236029,6.261330,1.580401,0.903957,24.137611,15.589334,0.007507,0.191340,0.156250,0.010417,24.754585,47.403113,0.320828,6.266547
2025-06-04,230.726101,28.221753,6.512869,1.602481,0.901393,26.763867,14.991984,0.023167,-0.110050,0.125000,0.012500,24.124967,49.540634,0.308253,6.528751


In [14]:
inf_data = inf_data.reset_index(drop=False)
inf_data.head(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
0,2024-01-01,229.442065,23.274581,5.33817,1.335934,0.904576,24.128362,15.61487,0.04442,-0.005502,0.072917,0.020833,24.458245,49.931362,0.307196,5.341409


In [16]:
n = len(inf_data)
inf_training_data = inf_data[:n-1]
inf_training_data.tail(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
519,2025-06-03,230.035298,27.236029,6.26133,1.580401,0.903957,24.137611,15.589334,0.007507,0.19134,0.15625,0.010417,24.754585,47.403113,0.320828,6.266547


In [20]:
inf_set = inf_data[n-10:]
inf_set.head(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
511,2025-05-26,230.123679,26.533685,6.112369,1.552541,0.898765,25.205411,14.641372,0.048538,0.193632,0.104167,0.041667,24.879234,50.752091,0.298928,6.086054
